In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification
from transformers import DistilBertForSequenceClassification
from transformers import AutoTokenizer
from transformers import BertConfig, BertModel
from transformers.modeling_outputs import SequenceClassifierOutput
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from torch.utils.tensorboard import SummaryWriter
import copy
import time

class TokenData(Dataset):
    def __init__(self, data, tokenizer):
        self.tokenizer = tokenizer
        # data = (train_x, train_tokens, train_y)#
        self.text_data = data[0]
        self.labels = data[1]

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        
        text = str(self.text_data[idx])
        label = 0 if self.labels[idx] == 1 else 1
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )

        # Return input_ids, attention_mask, and label
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = label
        return item
        
def regexPreProcessing(data):
    data.text = data.text.astype('str')
    data.text = data.text.str.lower()
    data.text = data.text.replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)                      # remove emails
    data.text = data.text.replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)    # remove IP address
    data.text = data.text.str.replace('[^\w\s]','')                                                       # remove special characters
    data.text = data.text.replace('\d', '', regex=True)
    return list(data.text), list(data.label)
    

2025-04-15 17:03:18.887336: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744729398.906238   60406 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744729398.911970   60406 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744729398.927121   60406 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744729398.927139   60406 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744729398.927141   60406 computation_placer.cc:177] computation placer alr

In [6]:
# For bert transformer #
# Code for provided datasets using amazon reviews #

"""data = pd.read_csv("amazon_cells_labelled.txt", delimiter='\t', header=None)
data = pd.read_csv("amazon_cells_labelled_LARGE_25K.txt", delimiter='\t', header=None)#
data.columns = ['Sentence', 'Class']
data['index'] = data.index                                          # add new column index
columns = ['index', 'Class', 'Sentence']
data = preprocess_pandas(data, columns)                             # pre-process
training_data, validation_data, training_labels, validation_labels = train_test_split( # split the data into training, validation, and test splits
    data['Sentence'].values.astype('U'),
    data['Class'].values.astype('int32'),
    test_size=0.10,
    random_state=0,
    shuffle=True
)
training_data = list(training_data)
testing_data = list(validation_data)
training_labels = list(training_labels)
testing_labels = list(validation_labels)"""

# Code for twitter dataset #

"""data = pd.read_csv("twitter_training.csv", delimiter=',', header=None)#
data.columns = ['id', 'context', 'Sentence', 'Class']
data['index'] = data.index                                          # add new column index
columns = ['id', 'context' ,'Class', 'Sentence', 'index']                      # pre-process
training_data, validation_data, training_labels, validation_labels = train_test_split( # split the data into training, validation, and test splits
    data['Sentence'].values.astype('U'),
    data['Class'].values.astype('U'),
    test_size=0.10,
    random_state=0,
    shuffle=True
)
training_data = list(training_data)
validation_data = list(validation_data)
training_labels = list(training_labels)
training_labels = [0 if v == 'Positive' else 1 if v == 'Negative' else 2 if v == 'Neutral' else 3 for v in training_labels]
validation_labels = list(validation_labels)
validation_labels = [0 if v == 'Positive' else 1 if v == 'Negative' else 2 if v == 'Neutral' else 3 for v in validation_labels]

data = pd.read_csv("twitter_test.csv", delimiter=',', header=None)#
data.columns = ['id', 'context', 'Class', 'Sentence']
data['index'] = data.index                                          # add new column index
columns = ['id', 'context' ,'Class', 'Sentence', 'index']     

testing_data = list(data['Sentence'])
testing_labels = list(data['Class'])
testing_labels = [0 if v == 'Positive' else 1 if v == 'Negative' else 2 if v == 'Neutral' else 3 for v in testing_labels]"""

"""# Code for hugging face dataset - multi category sentiment analysis

data = pd.read_csv("train_df.csv", delimiter=',')
training_data, training_labels = regexPreProcessing(data) 


data = pd.read_csv("val_df.csv", delimiter=',')
validation_data, validation_labels = regexPreProcessing(data)

data = pd.read_csv("test_df.csv", delimiter=',')
testing_data, testing_labels = regexPreProcessing(data)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
train_data = (training_data, tokenizer(training_data, padding = True, truncation=True, return_tensors="pt"), training_labels)
val_data = (validation_data, tokenizer(validation_data, padding = True,  truncation=True, return_tensors="pt"), validation_labels)
test_data = (testing_data, tokenizer(testing_data, padding = True, truncation=True, return_tensors="pt"), testing_labels)

batch_size = 20

train_dataset = TokenData(train_data, train = True)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

val_dataset = TokenData(val_data, train = True)

val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

test_dataset = TokenData(test_data ,train = False)

test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

print("training_loader_size:", len(train_loader), "validation_loader_size:", len(val_loader), "test_loader_size:", len(test_loader))"""

# Code for massive dataset #

batch_size = 150
data = pd.read_csv("train_massive.csv", delimiter=',', header=None)
data.columns = ['Class', 'Sentence', 'Body']
train_data = list(data['Sentence'])
train_labels = list(data['Class'])
train_data = (train_data, train_labels)
# Regular BERT uncased
#train_val_dataset = TokenData(train_data, tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased"))

# DistilBERT uncased
train_val_dataset = TokenData(train_data, tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased"))

train_dataset, val_dataset = torch.utils.data.random_split(train_val_dataset, [0.8, 0.2])

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

data = pd.read_csv("test_massive.csv", delimiter=',', header=None)
data.columns = ['Class', 'Sentence', 'Body']
testing_data = list(data['Sentence'])
testing_labels = list(data['Class'])
test_data = (testing_data, testing_labels)

# Regular BERT uncased
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#test_dataset = TokenData(test_data, tokenizer = tokenizer)

# DistilBERT
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
test_dataset = TokenData(test_data, tokenizer = tokenizer)

test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)
print(next(enumerate(test_loader))[1].keys(), len(next(enumerate(test_loader))[1]))
print(tokenizer.convert_ids_to_tokens(next(enumerate(test_loader))[1]['input_ids'][0], skip_special_tokens=False))
print(next(enumerate(test_loader))[1]['input_ids'][0])
print("training_loader_size:", len(train_loader), "validation_loader_size:", len(val_loader), "test_loader_size:", len(test_loader))


dict_keys(['input_ids', 'attention_mask', 'labels']) 3
['[CLS]', 'avoid', 'like', 'the', 'plague', '!', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [16]:
# Regular BERT training loop #

def train_model_bert(model, criterion, optimizer, train_loader, val_loader, num_epochs, batch_size, device):
    # For each epoch
    tensorboard_logging_path = "/tf/logs/BertTransformer_Massive2" + str(time.time())
    writer = SummaryWriter(log_dir=tensorboard_logging_path)
    best_validation_loss = float("inf")
    best_training_loss = float("inf")
    best_training_accuracy = 0
    best_validation_accuracy = 0
    for i in range(num_epochs):
        
        model.train()
        total_validation_loss = 0
        total_training_loss = 0 
        correct_train = 0
        correct_validation = 0
        for batch_nr, batch in enumerate(train_loader):
            
            input_ids, token_type_ids, attention_mask, labels = batch.values()
            
            input_ids = input_ids.to(device)
            token_type_ids = token_type_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.long().to(device)
            output = model(input_ids = input_ids, attention_mask = attention_mask) 
            prediction = output.logits
            training_loss = criterion(prediction, labels)
            writer.add_scalar('Loss Batch / Train', training_loss, i*batch_size + batch_nr)
            writer.flush()
            total_training_loss += training_loss.item()
            optimizer.zero_grad()
            
            training_loss.backward()
           
            optimizer.step() 
                
            #Print the epoch, batch, and loss
            print(
                '\rEpoch {} [{}/{}] - Loss: {}'.format(
                    i+1, batch_nr+1, len(train_loader), training_loss
                ),
                end=''
            )
        print()
            
        writer.add_scalar('Loss/Train', total_training_loss, i)
        
        model.eval()
        with torch.no_grad():
            for validation_nr, batch in enumerate(val_loader):
                
                input_ids, token_type_ids, attention_mask, labels = batch.values()
            
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.long().to(device)
                
                output = model(input_ids = input_ids, attention_mask = attention_mask) 
                prediction = output.logits
                
                validation_loss = criterion(prediction, labels)
                writer.add_scalar('Loss Batch / Validation', validation_loss, i*batch_size + validation_nr)
                writer.flush()
                total_validation_loss += validation_loss.item()
                
            if total_validation_loss < best_validation_loss:
                torch.save(model, "./models/BertTransformer_Massive2")
                best_validation_loss = total_validation_loss
        del input_ids, token_type_ids, attention_mask, labels, prediction, output, validation_loss, training_loss
        writer.add_scalar("Loss/Validation", total_validation_loss, i)
        writer.flush()
    
    writer.close()
    return model

In [25]:
# Distil BERT training loop #

def train_model_destil(model, criterion, optimizer, train_loader, val_loader, num_epochs, batch_size, device):
    # For each epoch
    tensorboard_logging_path = "/tf/logs/BertTransformer_MassiveDistil1" + str(time.time())
    writer = SummaryWriter(log_dir=tensorboard_logging_path)
    best_validation_loss = float("inf")
    best_training_loss = float("inf")
    best_training_accuracy = 0
    best_validation_accuracy = 0
    for i in range(num_epochs):
        
        model.train()
        total_validation_loss = 0
        total_training_loss = 0 
        correct_train = 0
        correct_validation = 0
        for batch_nr, batch in enumerate(train_loader):
            
            input_ids, attention_mask, labels = batch.values()
            
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.long().to(device)
            output = model(input_ids = input_ids, attention_mask = attention_mask) 
            prediction = output.logits
            training_loss = criterion(prediction, labels)
            writer.add_scalar('Loss Batch / Train', training_loss, i*batch_size + batch_nr)
            writer.flush()
            total_training_loss += training_loss.item()
            optimizer.zero_grad()
            
            training_loss.backward()
           
            optimizer.step() 
                
            #Print the epoch, batch, and loss
            print(
                '\rEpoch {} [{}/{}] - Loss: {}'.format(
                    i+1, batch_nr+1, len(train_loader), training_loss
                ),
                end=''
            )
        print()
            
        writer.add_scalar('Loss/Train', total_training_loss, i)
        
        model.eval()
        with torch.no_grad():
            for validation_nr, batch in enumerate(val_loader):
                
                input_ids, attention_mask, labels = batch.values()
            
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.long().to(device)
                
                output = model(input_ids = input_ids, attention_mask = attention_mask) 
                prediction = output.logits
                
                validation_loss = criterion(prediction, labels)
                writer.add_scalar('Loss Batch / Validation', validation_loss, i*batch_size + validation_nr)
                writer.flush()
                total_validation_loss += validation_loss.item()
                
            if total_validation_loss < best_validation_loss:
                torch.save(model, "./models/BertTransformer_MassiveDistil1")
                best_validation_loss = total_validation_loss
        del input_ids, attention_mask, labels, prediction, output, validation_loss, training_loss
        writer.add_scalar("Loss/Validation", total_validation_loss, i)
        writer.flush()
    
    writer.close()
    return model

In [4]:
# Regular BERT Base Uncased #

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2) # Pre-trained model
model.dropout.p = 0.3
optimizer = AdamW(model.parameters(), lr=5e-6, weight_decay = 0.01) # Optimization function
criterion = torch.nn.CrossEntropyLoss() # Loss function
epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # Transfer model to GPU if available
#trained_model = train_model_bert(model, criterion, optimizer, train_loader, val_loader, epochs, batch_size, device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [28]:
# DistilBERT Uncased #

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 2) # Pre-trained model
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay = 0.01) # Optimization function
criterion = torch.nn.CrossEntropyLoss() # Loss function
epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # Transfer model to GPU if available
trained_model = train_model_destil(model, criterion, optimizer, train_loader, val_loader, epochs, batch_size, device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 [19200/19200] - Loss: 0.23904785513877873


UnboundLocalError: cannot access local variable 'token_type_ids' where it is not associated with a value

In [31]:
model = torch.load("./models/BertTransformer_MassiveDistil1",weights_only = False)
#model = torch.load("./models/BertTransformerThingV1NoPreTraining",weights_only = False)
model.to(device)
correct = 0
total = 0
with torch.no_grad():
    for test_nr, batch in enumerate(test_loader):
                
        input_ids, attention_mask, labels = batch.values()
    
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        output = model(input_ids = input_ids, attention_mask = attention_mask)
        
        logits = output.logits
    
        correct += (logits.argmax(1) == labels).sum().item())
        total += labels.size(0)
        print('\rProgress {}'.format(test_nr), end = '')
print("Testing accuracy: ",correct/(total))

Progress 2666Testing accuracy:  0.8964425
